In [1]:
import csv
from itertools import product
from pandas import read_excel
import numpy as np
import pandas as pd
import os

# Number of pairs
dataset_size = 7000 


# Give the location of the files
directory = 'datasets_labeled/'

loc_generated_devset = ('datasets/dev_set.csv')
loc_generated_trainset = ('datasets/train_set.csv')
loc_generated_testset = ('datasets/test_set.csv')


In [ ]:


def get_all_pairs_from_file(directory, filename):
    term = filename.split('.')[0]
    df_rawgroups = read_excel(directory + filename, sheet_name="groups", index_col=None, header=None, engine='openpyxl')
    df_rawgroups = df_rawgroups.drop(df_rawgroups.columns[[0, 1]], axis=1)  # removing first and third unwanted column
    df_rawgroups = df_rawgroups.drop([0, 1])  # removing first and third unwanted column

    df_group_similarities = read_excel(directory + filename, sheet_name="group similarities", index_col=None, header=None, engine='openpyxl')
    df_group_similarities = df_group_similarities.drop(0, axis=0)
    df_group_similarities = df_group_similarities.drop(0, axis=1)

    df_definition = df_rawgroups[df_rawgroups.columns[range(0, df_rawgroups.shape[1]-1, 4)]]

    groups = []
    for column in df_definition.columns:
        group = []
        for definition in df_definition[column]: 
            if not pd.isna(definition): group.append(definition)
        groups.append(group)    
        pairs = []

    for grp_index1, group1 in enumerate(groups):
        for definition1 in group1:
            for grp_index2, group2 in enumerate(groups):
                for definition2 in group2:
                    if definition1 is not definition2:
                        sim = df_group_similarities.iloc[grp_index1,grp_index2]
                        if type(sim) is str: sim = sim.upper() 
                        if len(str(sim)) == 1 and (grp_index1 <= grp_index2) and not pd.isna(sim): 
                            pairs.append([term, term + str(grp_index1), term + str(grp_index2), definition1, definition2, sim])
    return pairs

pairs = []                  
for filename in os.listdir(directory):
    if not filename.startswith("~") and filename.endswith(".xlsx"):
        print(os.path.join(directory, filename))
        pairs.extend(get_all_pairs_from_file(directory, filename))
    else:
        continue

df_pairs = pd.DataFrame(pairs, columns=["term", "grp_id1", "grp_id2", "definition1", "definition2", "sim"])

df_pairs

In [ ]:
print(90*'=')
print("Datenset zusammenstellen: Möglichst ausgeglichene Anzahl von similarity labels,\nBenennungen und Begriffsgruppen erstellen")
print(90*'=' + "\n")

df = df_pairs.copy(deep=True)
df.insert(df.shape[1], "count_sim", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_term", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_grpid1", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_grpid2", [0]*df.shape[0], True) 
df.insert(df.shape[1], "count_def", [0]*df.shape[0], True) 

df_dataset = pd.DataFrame(columns=df.columns)

for i in range (0, dataset_size):
    #erstes Paar auswählen und entfernen
    select = df.iloc[0]
    df_dataset = df_dataset.append(select)
    df.drop(select.name, inplace=True, errors='raise')
    
    #eigenschaften hochzählen
    df.loc[df.sim == select.sim, 'count_sim'] += 1
    df.loc[df.term == select.term, 'count_term'] += 1
    df.loc[df.grp_id1 == select.grp_id1, 'count_grpid1'] += 1
    df.loc[df.grp_id1 == select.grp_id2, 'count_grpid2'] += 1
    df.loc[df.grp_id2 == select.grp_id1, 'count_grpid1'] += 1
    df.loc[df.grp_id2 == select.grp_id2, 'count_grpid2'] += 1
    df.loc[df.definition1 == select.definition1, 'count_def'] += 1
    df.loc[df.definition1 == select.definition2, 'count_def'] += 1
    df.loc[df.definition2 == select.definition1, 'count_def'] += 1
    df.loc[df.definition2 == select.definition2, 'count_def'] += 1
    
    #sortieren, so dass Paare mit Eigenschaften, die am seltensten vertreten sind, oben stehen 
    df.sort_values(by=['count_sim','count_def','count_grpid1','count_grpid2','count_term'], inplace=True)

    
#Verteilung ausgeben
print(df_dataset.groupby(['sim']).size())
print('')
print(df_dataset.groupby(['term']).size())




In [4]:
print(90*'=')
print("Datenset speichern")
print(90*'=' + "\n")
from sklearn.model_selection import train_test_split

testset_terms = ["domain", "hazard", "group", "event"]

train_df = df_dataset[~df_dataset["term"].isin(testset_terms)]
test_df = df_dataset[df_dataset["term"].isin(testset_terms)]
test_df.to_csv(loc_generated_testset)


train, dev = train_test_split(train_df, test_size=0.2)
train.to_csv(loc_generated_trainset)
dev.to_csv(loc_generated_devset)

Datenset speichern



In [ ]:
for term in df_dataset["term"].unique():
    df_dataset[df_dataset["term"] == term].to_csv("datasets_by_term/"+term+".csv")

In [ ]:
test_df

In [ ]:
train_df[train_df["term"]=="client"]